### Final Project - Bryan Becerra

In [9]:
import tkinter as tk  
from tkinter import ttk  
import sqlite3  # Importing sqlite3 for database operations

# Setting up the database
def setup_database():
    # Connecting to the SQLite database (creating a file if it does not exist for new user)
    conn = sqlite3.connect("unit_converter.db")
    cursor = conn.cursor()  # Creating a cursor to execute SQL commands
    # Creating a table for storing conversion records if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS conversions (
            id INTEGER PRIMARY KEY,  -- Setting a unique ID for each record
            input_value REAL,  -- Storing the user input value
            conversion_type TEXT,  -- Storing the type of conversion (Length, Weight, etc.)
            specific_conversion TEXT,  -- Storing the specific conversion (e.g., Meters to Feet)
            result REAL  -- Storing the conversion result
        )
    """)
    conn.commit()  # Committing changes to the database
    conn.close()  # Closing the connection

# Function for performing unit conversion based on user input
def convert_units():
    try:
        # Getting the input value from the user
        value = float(entry.get())
        # Getting the type of conversion selected by the user
        conversion_type = conversion_type_combobox.get()
        # Getting the specific conversion selected by the user
        specific_conversion = conversion_combobox.get()

        # Performing the selected conversion
        if conversion_type == "Length":
            if specific_conversion == "Meters to Feet":
                result = value * 3.28084  # Converting meters to feet
            elif specific_conversion == "Feet to Meters":
                result = value / 3.28084  # Converting feet to meters
        elif conversion_type == "Weight":
            if specific_conversion == "Kilograms to Pounds":
                result = value * 2.20462  # Converting kilograms to pounds
            elif specific_conversion == "Pounds to Kilograms":
                result = value / 2.20462  # Converting pounds to kilograms
        elif conversion_type == "Temperature":
            if specific_conversion == "Celsius to Fahrenheit":
                result = (value * 9/5) + 32  # Converting Celsius to Fahrenheit
            elif specific_conversion == "Fahrenheit to Celsius":
                result = (value - 32) * 5/9  # Converting Fahrenheit to Celsius
        else:
            # Handling invalid or incomplete selections
            result_label.config(text="Please select valid options.")
            return

        # Displaying the conversion result in the result label
        result_label.config(text=f"Result: {result:.2f}")
        # Saving the conversion record to the database
        save_to_database(value, conversion_type, specific_conversion, result)
    except ValueError:
        # Handling non-numeric values
        result_label.config(text="Invalid input. Please enter a number.")

# Function for saving the conversion record to the database
def save_to_database(input_value, conversion_type, specific_conversion, result):
    # Connecting to the SQLite database
    conn = sqlite3.connect("unit_converter.db")
    cursor = conn.cursor()
    # Inserting the record into the conversions table
    cursor.execute("""
        INSERT INTO conversions (input_value, conversion_type, specific_conversion, result)
        VALUES (?, ?, ?, ?)
    """, (input_value, conversion_type, specific_conversion, result))
    conn.commit()  # Committing the insertion
    conn.close()  # Closing the connection

# Function for viewing all results
def view_records():
    # Fetching all records from the conversions table
    conn = sqlite3.connect("unit_converter.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM conversions") # Executes the pull from the db
    records = cursor.fetchall()  # Getting all rows as a list of tuples
    conn.close()  # Closing the connection

    # Creating a new window to display the records
    records_window = tk.Toplevel(root)
    records_window.title("Conversion Records")
    records_window.geometry("450x350")  # Setting window size
    records_window.configure(bg="#eaf6f6")  # Setting light aqua background color

    # Creating a listbox to display the records
    listbox = tk.Listbox(records_window, width=55, height=15, font=("Courier New", 12))
    listbox.pack(pady=20)

    # Inserting each record into the listbox
    for record in records:
        listbox.insert(tk.END, f"ID: {record[0]}, Value: {record[1]}, {record[2]} ({record[3]}), Result: {record[4]:.2f}")

# Handling the app close-event to clear the database
def on_app_close():
    root.destroy()  # Closing the main application window

# Setting up the main application window
root = tk.Tk()
root.title("Unit Converter App with Database")  # Setting the window title
root.geometry("550x620")  # Setting the window size
root.configure(bg="#d4ebf2")  # Setting light blue background color

# Adding the main header to the app
header = tk.Label(root, text="Unit Converter App", font=("Helvetica", 26, "bold"), bg="#d4ebf2", fg="#004466")
header.pack(pady=10)

# Adding a description of what the app does
description = tk.Label(
    root,
    text="Easily convert between units of length, weight, or temperature.\n"
         "Store and view past conversions for quick reference.",
    font=("Helvetica", 12),
    bg="#d4ebf2",
    fg="#004466",
    justify="center"
)
description.pack(pady=10)

# Adding an underheader with the developer's name
underheader = tk.Label(root, text="Developed by Bryan Becerra", font=("Helvetica", 14), bg="#d4ebf2", fg="#006680")
underheader.pack(pady=5)

# Adding an input field for the user to enter a value
tk.Label(root, text="Enter value:", bg="#d4ebf2", font=("Helvetica", 14)).pack(pady=5)
entry = tk.Entry(root, font=("Helvetica", 12), width=20)
entry.pack(pady=5)

# Adding a combobox for selecting the conversion type
tk.Label(root, text="Select Conversion Type:", bg="#d4ebf2", font=("Helvetica", 14)).pack(pady=5)
conversion_type_combobox = ttk.Combobox(root, values=["Length", "Weight", "Temperature"], font=("Helvetica", 12))
conversion_type_combobox.pack(pady=5)

# Adding a combobox for selecting the specific conversion
tk.Label(root, text="Select Conversion:", bg="#d4ebf2", font=("Helvetica", 14)).pack(pady=5)
conversion_combobox = ttk.Combobox(root, font=("Helvetica", 12))
conversion_combobox.pack(pady=5)

# Updating specific conversion options when a conversion type is selected
def populate_conversions(event):
    conversion_type = conversion_type_combobox.get()
    if conversion_type == "Length":
        conversion_combobox.config(values=["Meters to Feet", "Feet to Meters"])
    elif conversion_type == "Weight":
        conversion_combobox.config(values=["Kilograms to Pounds", "Pounds to Kilograms"])
    elif conversion_type == "Temperature":
        conversion_combobox.config(values=["Celsius to Fahrenheit", "Fahrenheit to Celsius"])
    conversion_combobox.set("")  # Clearing any previous selection

conversion_type_combobox.bind("<<ComboboxSelected>>", populate_conversions)

# Adding a button to perform the conversion
convert_button = tk.Button(root, text="Convert", command=convert_units, font=("Helvetica", 14), bg="#008080", fg="white", width=12, height=2)
convert_button.pack(pady=20)

# Adding a button to view all saved records
view_records_button = tk.Button(root, text="View Records", command=view_records, font=("Helvetica", 14), bg="#004466", fg="white", width=12, height=2)
view_records_button.pack(pady=10)

# Adding a label to display the conversion result
result_label = tk.Label(root, text="Result: ", bg="#d4ebf2", font=("Helvetica", 16, "bold"), fg="#004466")
result_label.pack(pady=20)

# Binding closing the app to restarting what was recorded from a previous run of the app
root.protocol("WM_DELETE_WINDOW", on_app_close)

# Running the whole application
root.mainloop()
